In [1]:
#python 包
from PIL import Image
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import time

In [2]:
# 视频路径
route_video='./night_video/4.avi'

# 直接截取帧保存目录
direct_frames_target='./night_direct_images'

# 处理帧保存目录
route_target='./night_images'


# 生成帧目录
ge_frames_route='./ge_night_frames'

im_dir = './ge_night_frames'
video_dir = './ge_night_video/1.avi'

# 垂直方向的裁剪标准
x1=80
x2=750
y1=80
y2=1050

# 分隔区间数目
bins=18

# 抖动阈值
# 大约17秒处开始抖动
thres_shaking=5000

# 掉线阈值
# 大约1分07秒掉线，
thres_drop=20000

In [3]:
# 返回 (FRAME_COUNT,FRAME_FPS)
# 外部需要定义剪裁比例x1和x2
def generate_frames(route_video,route_target,direct_frames_target):
    # 生成帧
    # 需要视频完整路径和存放帧的目录
    vidcap=cv2.VideoCapture(route_video)
    # 获取总帧数
    FRAME_COUNT=vidcap.get(7)-1
    # 获取FPS
    FRAME_FPS=vidcap.get(5)
    
    success,image=vidcap.read()
    count=0
    success=True
    shape1=0
    while success:
        success,image=vidcap.read()
        
        if image is None:
            break
        shape1=image.shape
        res_direct_frames_target=direct_frames_target+'/frame'+str(count)+'.jpg'
        cv2.imwrite(res_direct_frames_target,image)
        image2=image[x1:x2,y1:y2]
        shape0=image2.shape
        image3=cv2.resize(image2,(int(shape0[1]/5),int(shape0[0]/5)),interpolation=cv2.INTER_AREA)
        blur=cv2.GaussianBlur(image3,(3,3),0)
        res_target=route_target+'/frame'+str(count)+'.jpg'
        cv2.imwrite(res_target,blur)
    
        #print('frame%d OK' % count)
        if cv2.waitKey(10)==27:
            break
        count+=1 
    
    print('Generate frames OK.')
    return (int(FRAME_COUNT),int(FRAME_FPS),shape1)

In [4]:
# param:
# route_video:视频路径
# route_target:帧保存目录
# 每一秒随机选random_test_frames_num帧 ,默认10
# 对照帧随机从一秒选random_contrast_num帧，默认3

# 外部需要定义:
# 剪裁比例x1和x2
# 分隔区间数目bins
def handle_frame(route_video,route_target,direct_frames_target,ge_frames_route,random_test_frames_num=10,random_contrast_num=3):
    #生成帧（已经滤波）
    #返回 FRAME_COUNT,FRAME_FPS
    start_time=time.time()
    FRAME_COUNT,FRAME_FPS,FRAME_SHAPE=generate_frames(route_video,route_target,direct_frames_target)
    #FRAME_COUNT=1890
    #FRAME_FPS=25
    end_time=time.time()
    print('Generate frames with %fs' % (end_time-start_time))

    # 视频总时长/s
    VIDEO_DURATION=int(FRAME_COUNT/FRAME_FPS)
    
    #第一帧默认为标准帧
    
    
    #正常的秒数（只保存最新两个）
    contrast_seconds=list(np.zeros(2))
    #上述数组已存储秒的个数
    contrast_seconds_filled_size=0
    
    #1-(fps-2)的数列
    fps_list=list(range(0,FRAME_FPS-1))
    
    #1个循环处理1秒
    #处理测试帧：
    t_blur=list(np.zeros(random_test_frames_num))
    t_gray=list(np.zeros(random_test_frames_num))
    t_thres=list(np.zeros(random_test_frames_num))
    t_sobelx=list(np.zeros(random_test_frames_num))
    t_sobely=list(np.zeros(random_test_frames_num))
    t_phase=list(np.zeros(random_test_frames_num))
    t_list=list(np.zeros(random_test_frames_num))
    t_over=list(np.zeros(random_test_frames_num))
    #每一项都是包含bins个的区间高度
    t_patches=list(np.zeros(random_test_frames_num))
    
    #生成帧序号：
    ge_index=0
    
    for i in range(VIDEO_DURATION):
        start_time=time.time()
        #获得一秒的最后一帧
        curr_sec_last_frame=(i+1)*FRAME_FPS-1
        #获得一秒的随机的random_test_frames_num的帧的标号
        random_test_frames_order=random.sample(fps_list,random_test_frames_num)
        
        #获得对应的random_test_frames_num个帧的完整路径
        random_test_frames_routes=list(np.zeros(random_test_frames_num))
        for j in range(random_test_frames_num):
            random_test_frames_routes[j]=route_target+'/frame'+str(curr_sec_last_frame-random_test_frames_order[j])+'.jpg'
            
        
        #循环处理测试帧
        time_start_1=time.time()
        for j in range(random_test_frames_num):
            t_blur[j] = cv2.imread(random_test_frames_routes[j])
            t_gray[j] = cv2.cvtColor(t_blur[j],cv2.COLOR_BGR2GRAY)
            t_thres[j] = cv2.adaptiveThreshold(t_gray[j],255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,3,2)
            t_sobelx[j] = cv2.Sobel(t_thres[j],cv2.CV_64F,1,0,ksize=3)#1,0表示只在x方向求一阶导数
            t_sobely[j] = cv2.Sobel(t_thres[j],cv2.CV_64F,0,1,ksize=3)#0,1表示只在y方向求一阶导数
            t_phase[j] = cv2.phase(t_sobelx[j],t_sobely[j],angleInDegrees=True)
            t_list[j] = list(t_phase[j].reshape(-1))
            t_over[j]=[]
            for item in t_list[j]:
                if item >= 180.0:
                    t_over[j].append(item-180.0)
            _,_,patches=plt.hist(t_over[j],bins,[0,180])
            t_patches[j]=[]
            for l in range(bins):
                t_patches[j].append(patches[l].get_height())
                
        #开始与对照帧比对
        
        # 与最近的两对照秒比较
        #获得对应的random_contrast_num个帧的完整路径
        random_contrast_frames_routes=[]
            
        random_contrast_frames_order1=random.sample(fps_list,random_contrast_num)
        random_contrast_frames_order2=random.sample(fps_list,random_contrast_num)
            
        
        if contrast_seconds_filled_size==2:
            for j in range(random_contrast_num):
                random_contrast_frames_routes.append(route_target+'/frame'+str(((contrast_seconds[0]+1)*FRAME_FPS-1)-random_contrast_frames_order1[j])+'.jpg')
            for j in range(random_contrast_num):
                random_contrast_frames_routes.append(route_target+'/frame'+str(((contrast_seconds[1]+1)*FRAME_FPS-1)-random_contrast_frames_order2[j])+'.jpg')
        
        elif contrast_seconds_filled_size==1:
            for j in range(random_contrast_num):
                random_contrast_frames_routes.append(route_target+'/frame'+str(((contrast_seconds[0]+1)*FRAME_FPS-1)-random_contrast_frames_order1[j])+'.jpg')
        
        else:
            random_contrast_frames_routes.append(route_target+'/frame0.jpg')
         
        # 实际用到的对照帧个数
        len_contrast_frames=len(random_contrast_frames_routes)
        #print('对照帧数：',len_contrast_frames)    
         
        time_end_1=time.time()
        #print('time_1: %f' % (time_end_1-time_start_1))
        
        time_start_2=time.time()
        
        blur2=list(np.zeros(len_contrast_frames))
        gray2=list(np.zeros(len_contrast_frames))
        thres2=list(np.zeros(len_contrast_frames))
        sobelx2=list(np.zeros(len_contrast_frames))
        sobely2=list(np.zeros(len_contrast_frames))
        phase2=list(np.zeros(len_contrast_frames))
        list2=list(np.zeros(len_contrast_frames))
        over2=list(np.zeros(len_contrast_frames))
        #每一项都是包含bins个的区间高度
        patches2=list(np.zeros(len_contrast_frames))
        
        for j in range(len_contrast_frames):
            blur2[j] = cv2.imread(random_contrast_frames_routes[j])
            gray2[j] = cv2.cvtColor(blur2[j],cv2.COLOR_BGR2GRAY)
            thres2[j] = cv2.adaptiveThreshold(gray2[j],255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,3,2)
            sobelx2[j] = cv2.Sobel(thres2[j],cv2.CV_64F,1,0,ksize=3)#1,0表示只在x方向求一阶导数
            sobely2[j] = cv2.Sobel(thres2[j],cv2.CV_64F,0,1,ksize=3)#0,1表示只在y方向求一阶导数
            phase2[j] = cv2.phase(sobelx2[j],sobely2[j],angleInDegrees=True)
            list2[j] = list(phase2[j].reshape(-1))
            over2[j]=[]
            for item in list2[j]:
                if item >= 180.0:
                    over2[j].append(item-180.0)
            _,_,patches=plt.hist(over2[j],bins,[0,180])
            patches2[j]=[]
            for l in range(bins):
                patches2[j].append(patches[l].get_height())    
       
    
        diff_avg=list(np.zeros(random_test_frames_num))
        for j in range(random_test_frames_num):
            for k in range(len_contrast_frames):
                
                diff=0.0
                for l in range(bins):
                    diff+=np.square(patches2[k][l]-t_patches[j][l])
                
                diff_avg[j]+=diff/(len_contrast_frames)
        
        # 计算各测试帧的情况
        shaking_count=0
        drop_count=0
        for j in range(random_test_frames_num):
            if diff_avg[j]>thres_drop:
                drop_count+=1
                shaking_count+=1
            elif diff_avg[j]>thres_shaking:
                shaking_count+=1
                
        
        res_diff=np.sum(diff_avg)/len(diff_avg)
        
        time_end_2=time.time()
        #print('time_2: %f' % (time_end_2-time_start_2))
        
        time_start_3=time.time()
        
        #取当前秒最后一帧图
        temp_route_str=route_target+'/frame'+str(curr_sec_last_frame)+'.jpg'
        curr_last_frame=cv2.imread(temp_route_str)
        
        #输出帧
        t_ge=list(np.zeros(random_test_frames_num))
        for j in range(random_test_frames_num):
            get_ge_str=direct_frames_target+'/frame'+str(curr_sec_last_frame-random_test_frames_order[j])+'.jpg'
            t_ge[j]=cv2.imread(get_ge_str)
        
        if contrast_seconds_filled_size==2:
            if drop_count>int(random_test_frames_num/2)-2:
                print('%ds DROP-LINE with difference %f' % (i,res_diff))
                for j in range(random_test_frames_num):
                    t_ge[j]=cv2.circle(t_ge[j],(50,50), 25, (0,0,255), -1)
                
            elif shaking_count>int(random_test_frames_num/2)-2:
                print("\033[0;31m%ds SHAKING with difference %f\033[0m" % (i,res_diff))
                #print('%ds SHAKING with difference %f' % (i,res_diff))
                for j in range(random_test_frames_num):
                    t_ge[j]=cv2.circle(t_ge[j],(50,50), 25, (0,255,255), -1)
                
            else:
                contrast_seconds[0]=contrast_seconds[1]
                contrast_seconds[1]=i
                print('%ds stable with difference %f' % (i,res_diff))
                for j in range(random_test_frames_num):
                    t_ge[j]=cv2.circle(t_ge[j],(50,50), 25, (0,255,0), -1)
                
        elif contrast_seconds_filled_size==1:
            if drop_count>int(random_test_frames_num/2)-2:
                print('%ds DROP-LINE with difference %f' % (i,res_diff))
                for j in range(random_test_frames_num):
                    t_ge[j]=cv2.circle(t_ge[j],(50,50), 25, (0,0,255), -1)
                #curr_last_frame=cv2.circle(curr_last_frame,(20,20), 10, (0,0,255), -1)
            elif shaking_count>int(random_test_frames_num/2)-2:
                print('%ds SHAKING with difference %f' % (i,res_diff))
                for j in range(random_test_frames_num):
                    t_ge[j]=cv2.circle(t_ge[j],(50,50), 25, (0,255,255),-1)
            else:
                contrast_seconds[1]=i
                contrast_seconds_filled_size+=1
                print('%ds stable with difference %f' % (i,res_diff))
                
        else:
            if drop_count>int(random_test_frames_num/2)-2:
                print('%ds DROP-LINE with difference %f' % (i,res_diff))
                for j in range(random_test_frames_num):
                    t_ge[j]=cv2.circle(t_ge[j],(50,50), 25, (0,0,255), -1)
                #curr_last_frame=cv2.circle(curr_last_frame,(20,20), 10, (0,0,255), -1)
            elif shaking_count>int(random_test_frames_num/2)-2:
                print('%ds SHAKING with difference %f' % (i,res_diff))
                for j in range(random_test_frames_num):
                    t_ge[j]=cv2.circle(t_ge[j],(50,50), 25, (0,255,255),-1)
            else:
                contrast_seconds[0]=i
                contrast_seconds_filled_size+=1
                print('%ds stable with difference %f' % (i,res_diff))
            
        print('最近两秒：%ds/%ds' % (contrast_seconds[0],contrast_seconds[1]))
        
        
        for j in range(random_test_frames_num):
            ge_frame_route=ge_frames_route+'/'+str(ge_index)+'.jpg'
            cv2.imwrite(ge_frame_route,t_ge[j])
            ge_index+=1
            
        time_end_3=time.time()
        #print('time_3:%f' % (time_end_3-time_start_3))
        
        end_time=time.time()
        print(' (duration: %fs)' % (end_time-start_time))
        
    return (VIDEO_DURATION,FRAME_SHAPE)

In [5]:
ge_frames_num,ge_frames_shape=handle_frame(route_video,route_target,direct_frames_target,ge_frames_route)

Generate frames OK.
Generate frames with 21.773925s
0s stable with difference 2249.000000
最近两秒：0s/0s
 (duration: 0.851918s)
1s stable with difference 4432.933333
最近两秒：0s/1s
 (duration: 0.917485s)
2s SHAKING with difference 4977.133333
最近两秒：0s/1s
 (duration: 1.075647s)
3s SHAKING with difference 7036.533333
最近两秒：0s/1s
 (duration: 1.042567s)
4s SHAKING with difference 7336.266667
最近两秒：0s/1s
 (duration: 1.134820s)
5s SHAKING with difference 5018.200000
最近两秒：0s/1s
 (duration: 1.233567s)
6s stable with difference 3229.700000
最近两秒：1s/6s
 (duration: 1.291293s)
7s SHAKING with difference 5170.566667
最近两秒：1s/6s
 (duration: 1.436375s)
8s stable with difference 3871.166667
最近两秒：6s/8s
 (duration: 1.477376s)
9s stable with difference 4338.133333
最近两秒：8s/9s
 (duration: 1.599322s)
10s SHAKING with difference 5431.900000
最近两秒：8s/9s
 (duration: 1.817948s)
11s stable with difference 3756.300000
最近两秒：9s/11s
 (duration: 2.010662s)
12s SHAKING with difference 6556.133333
最近两秒：9s/11s
 (duration: 2.232995s)


In [6]:

fps = 10
num = ge_frames_num*10
img_size = ge_frames_shape
fourcc = cv2.VideoWriter_fourcc('M','J','P','G') #opencv3.0
videoWriter = cv2.VideoWriter(video_dir, fourcc, fps,(img_size[1],img_size[0]))

for i in range(num):
    #im_name = os.path.join(im_dir, str(i).zfill(6)+'.jpg')
    im_name=im_dir+'/'+str(i)+'.jpg'
    frame = cv2.imread(im_name)
    videoWriter.write(frame)
    print(im_name)

videoWriter.release()
print('Generate video finish')

./ge_night_frames/0.jpg
./ge_night_frames/1.jpg
./ge_night_frames/2.jpg
./ge_night_frames/3.jpg
./ge_night_frames/4.jpg
./ge_night_frames/5.jpg
./ge_night_frames/6.jpg
./ge_night_frames/7.jpg
./ge_night_frames/8.jpg
./ge_night_frames/9.jpg
./ge_night_frames/10.jpg
./ge_night_frames/11.jpg
./ge_night_frames/12.jpg
./ge_night_frames/13.jpg
./ge_night_frames/14.jpg
./ge_night_frames/15.jpg
./ge_night_frames/16.jpg
./ge_night_frames/17.jpg
./ge_night_frames/18.jpg
./ge_night_frames/19.jpg
./ge_night_frames/20.jpg
./ge_night_frames/21.jpg
./ge_night_frames/22.jpg
./ge_night_frames/23.jpg
./ge_night_frames/24.jpg
./ge_night_frames/25.jpg
./ge_night_frames/26.jpg
./ge_night_frames/27.jpg
./ge_night_frames/28.jpg
./ge_night_frames/29.jpg
./ge_night_frames/30.jpg
./ge_night_frames/31.jpg
./ge_night_frames/32.jpg
./ge_night_frames/33.jpg
./ge_night_frames/34.jpg
./ge_night_frames/35.jpg
./ge_night_frames/36.jpg
./ge_night_frames/37.jpg
./ge_night_frames/38.jpg
./ge_night_frames/39.jpg
./ge_night